<img src="https://pages.cnpem.br/capsuladaciencia/wp-content/uploads/sites/155/2022/10/Ilum.png" alt="Ilum - Escola de Ciência" width="200"/>

**Redes Neurais e Algoritmos Genéticos 2025**

**Prof. Dr. Daniel R. Cassar**

Rafael Dalacorte Erdmann (24017)

## Fera Formidável 9: A senha de tamanho variável

**Objetivo:** Resolver o problema da senha de forma que você não forneça a informação do tamanho da senha para a função que gera a população. Considere que a senha pode ser uma string de 1 até 30 caracteres.

**Dica:** A função objetivo terá que quantificar em sua métrica tanto se o candidato acertou as letras quanto se acertou o tamanho da senha.

**Dica 2:** Você pode criar diferentes estratégias de mutação, não precisa ser apenas uma! Quem sabe uma função de mutação pode alterar letras e a outra pode alterar o tamanho da senha? Ver o exercício “Praticamente um X-man!”.

**Dica 3:** Observe que você terá que pensar um pouco sobre como fará o cruzamento no caso de senhas de tamanhos diferentes. Quem sabe tenha que fazer alguma consideração adicional sobre quais são os valores possíveis para o ponto de corte…

### Introdução

O problema da senha não é nada mais que um problema de caixas não-binárias [1], onde há tantas caixas quanto dígitos da senha e dentro de cada caixa pode estar um caractere dentre os possíveis. Além disso, a função objetivo é a semelhança entre a senha do candidato e a senha real (ou melhor, a distância entre as duas, que deve ser minimizada). Nessa Fera, mais especificamente, o número de caixas correto não é conhecido inicialmente pelos indivíduos da população, que deverão encontrá-lo juntamente com os caracteres corretos ao longo das gerações.

Para resolvê-lo, utilizaremos um algoritmo genético com operadores pensados especificamente para o problema.

### Resolução

Para resolver o problema, foi necessário editar alguns operadores: cria_população, funcao_objetivo, funcao_cruzamento e funcao_mutacao. Cada um será explicado adiante, após a importação das funções e definição dos parâmetros do problema.

In [145]:
from random import seed 
from string import ascii_lowercase, ascii_uppercase, digits

from funcoes_feras import populacao_senha_variavel as cria_populacao
from funcoes_feras import funcao_objetivo_pop_senha_variavel as funcao_objetivo
from funcoes_feras import selecao_torneio_min as funcao_selecao
from funcoes_feras import cruzamento_ponto_simples_variavel as funcao_cruzamento
from funcoes_feras import mutacao_simples as funcao_mutacao1
from funcoes_feras import mutacao_tamanho as funcao_mutacao2

In [146]:
SENHA_ALEATORIA = 42
seed(SENHA_ALEATORIA)

SENHA = list("ResponsavelPeloQueCativas43")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits

TAMANHO_MIN_SENHA = 1
TAMANHO_MAX_SENHA = 30

TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
TAMANHO_TORNEIO = 3

#### População

Na geração da população inicial, foi necessário alterar a entrada de tamanho de senha para dois argumentos diferentes: o tamanho mínimo e máximo da senha (nesse problema definidos como 1 e 30, respectivamente). Com isso, o módulo `random` pode ser utilizado para gerar uma senha com tamanho incluído nesse intervalo, como 3 dígitos, 30 dígitos, etc.

In [147]:
populacao = cria_populacao(TAMANHO_POPULACAO, TAMANHO_MIN_SENHA, TAMANHO_MAX_SENHA, CARACTERES_POSSIVEIS)

for i in range(len(populacao)):
    print (i, "".join(populacao[i]))

0 hbVrpoiVgRV5IfLBcbfno
1 MbJmTPSIAoCLrZ3aW
2 kSBvrjn9Wvgfygw2wMqZcUDIh7
3 fJs1ON43xKmTe
4 Qo
5 sf2o3gyrDO1xkxwnQrS7RPeMO
6 IUpkDy
7 7OSJoRu1X
8 do0cZuzren68K4TunPFz46PDj
9 ipVJIqVLB
10 LzxoiGFfWd3hjOkYRBMeyyMDHqJ38
11 R
12 hR4IWrXPvhsBkDa9U4UqGWlG
13 g3Ot1OGMmjxWkI9X7H6aMuFbh7x41Z
14 pdp4K8ffUF
15 eWIXiiQE8JkqH3MB9n7IWUSmTtz
16 PxC5HChpoevbLJoLoaeTOd
17 e5c3veGp
18 QFnIiU74K
19 EpYEZAmggQBwBAD3UdR
20 PgdzUvZ3gpmmICiBlrDp3
21 eCZ32JgdPI1af7W2pkAFEn3z5dkyay
22 7YYDsBS9U
23 JQTFjmsn9dLVIdVuddLEG62Hkd
24 f2leMeR3pzh84KpLM
25 Nf
26 QLKHu7qnQTupqz
27 QPtDu
28 W7eaDNKgeInGqi7w4e4pxskC1ITtNZ
29 HaQ0Jt7Qg84iqh4gVJjrs
30 nTvnRO2qGFq562dfOB1r
31 av
32 iOqkVCJTBJahe84S5jIc1xLJj
33 ictx57Y3c5wnRp
34 gwXJ43ANVj77p3kZZl4Abl
35 7vY7AZQ3VZprkYSgy3c2Eom0
36 Dwt0Y3oobQmzvr3e9XrwPGzR1Iv8bh
37 qlL9qcgMBwUYuBMGhy5KmqcTBaH7Z
38 RU8VVQmxBe8Q6vNuQ2
39 U5tG
40 QAuzSsJimA
41 8yRV5lNKtzJ1atsnBYLMPu
42 CCRnGEY59YVkQfs
43 QONvf08WpRtoZmjbc
44 EN2XeDA4
45 KmTSyFzpjPSa5W3X4gXBo
46 Z9SHDd
47 p62hDiZDQHJMu8W5CN
48 U5G

#### Função Objetivo

Como função objetivo, neste problema o algoritmo deve considerar tanto se as letras estão corretas quanto se o tamanho da senha também está. Para isso, a função objetivo do problema da senha tradicional foi utilizado como base, mas adicionando-se a distância entre o tamanho da senha candidata e da senha verdadeira. Nesta etapa, tentei normalizar as distâncias para que a distância das letras tivesse o mesmo peso que a distância do tamanho, mas para esse problema é notável que fica muito melhor manter a distância das letras com mais peso.

Por quê? Pensemos na situação inversa: se a distância do tamanho for mais importante, o algoritmo primeiro encontrará o tamanho certo da senha, o que para uma senha de 25 dígitos significaria ter que continuar realizando cruzamentos e mutações em todas letras até que as 25 estivessem certas. Da maneira como o algoritmo aqui foi construído, ele tende a primeiro identificar as letras corretas e só então ir adicionando novos dígitos, o que restringe a região de busca, perdendo menos informação com mutações equivocadas, e agiliza o processo.

In [148]:
fitness = funcao_objetivo(populacao, SENHA) 

for i in range(len(fitness)):
    print(i, f"{fitness[i]:0.2f}", "".join(populacao[i]))

0 326.00 hbVrpoiVgRV5IfLBcbfno
1 420.00 MbJmTPSIAoCLrZ3aW
2 494.00 kSBvrjn9Wvgfygw2wMqZcUDIh7
3 330.00 fJs1ON43xKmTe
4 36.00 Qo
5 657.00 sf2o3gyrDO1xkxwnQrS7RPeMO
6 108.00 IUpkDy
7 245.00 7OSJoRu1X
8 658.00 do0cZuzren68K4TunPFz46PDj
9 262.00 ipVJIqVLB
10 591.00 LzxoiGFfWd3hjOkYRBMeyyMDHqJ38
11 26.00 R
12 540.00 hR4IWrXPvhsBkDa9U4UqGWlG
13 693.00 g3Ot1OGMmjxWkI9X7H6aMuFbh7x41Z
14 283.00 pdp4K8ffUF
15 797.00 eWIXiiQE8JkqH3MB9n7IWUSmTtz
16 439.00 PxC5HChpoevbLJoLoaeTOd
17 238.00 e5c3veGp
18 273.00 QFnIiU74K
19 494.00 EpYEZAmggQBwBAD3UdR
20 389.00 PgdzUvZ3gpmmICiBlrDp3
21 855.00 eCZ32JgdPI1af7W2pkAFEn3z5dkyay
22 280.00 7YYDsBS9U
23 618.00 JQTFjmsn9dLVIdVuddLEG62Hkd
24 404.00 f2leMeR3pzh84KpLM
25 30.00 Nf
26 316.00 QLKHu7qnQTupqz
27 95.00 QPtDu
28 639.00 W7eaDNKgeInGqi7w4e4pxskC1ITtNZ
29 590.00 HaQ0Jt7Qg84iqh4gVJjrs
30 553.00 nTvnRO2qGFq562dfOB1r
31 57.00 av
32 636.00 iOqkVCJTBJahe84S5jIc1xLJj
33 363.00 ictx57Y3c5wnRp
34 594.00 gwXJ43ANVj77p3kZZl4Abl
35 676.00 7vY7AZQ3VZprkYSgy3c2Eom0
36 77

#### Cruzamento

Como operador de cruzamento, aqui utilizei uma versão alterada do cruzamento de ponto simples: como os candidatos parentes podem ter tamanhos diferentes, o ponto de corte se baseia no menor pai. Tendo o ponto de corte, os filhos possuem os genes cruzados dos pais antes e após esse ponto.

Exemplo:

In [149]:
mae = list("TESTE")
pai = list("test")

filho_1, filho_2 = funcao_cruzamento(pai, mae, 1)
print("".join(filho_1), "".join(filho_2), sep="\n")

tESTE
Test


Foi necessário também definir um comportamento diferente para caso um dos parentes tenha apenas um dígito: ele é trocado com algum dígito do outro parente. Observe o exemplo:

In [150]:
mae = list("teste")
pai = list("A")

filho_1, filho_2 = funcao_cruzamento(pai, mae, 1)
print("".join(filho_1), "".join(filho_2), sep="\n")

t
tesAe


#### Mutação

Nessa etapa, observa-se que é essencial ter no mínimo dois operadores de mutação: um para alterar os dígitos e um para alterar o tamanho da senha. Sem qualquer um dos dois, o algoritmo não vai encontrar a resposta certa (e pelo critério de parada ser esse, ele entrará em um loop infinito).

Como mutação dos dígitos, utilizei a mutação simples, sem alteração alguma: ela seleciona um índice da senha e troca seu valor por outro dentre os possíveis. 

Para mutar o tamanho da senha, porém, foi necessário gerar um novo operador (`mutacao_tamanho`), que pode aumentar o tamanho em um (adicionando um dígito possível), ou diminuir o tamanho em um (excluindo o último dígito). Observe o exemplo abaixo:

In [151]:
pop_tamanho = [list("TESTE"), list("teste"), list("Tamanho")]

funcao_mutacao2(pop_tamanho, 1, list(CARACTERES_POSSIVEIS))
for i in pop_tamanho:
    print("".join(i))

TESTEX
testeo
TamanhoR


#### Aplicação dos operadores

Podemos, então, rodar o algoritmo com todos os operadores novos conforme loop `while` adiante:

In [152]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0:
    
    # Seleção
    fitness = funcao_objetivo(populacao, SENHA)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
    
    # Encerramento
    populacao = proxima_geracao
    geracao += 1
    
    fitness = funcao_objetivo(populacao, SENHA)
    menor_fitness_observado = min(fitness)
    
    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 R
28 Re
46 Res
128 Resp
152 Respo
200 Respon
223 Respons
249 Responsa
264 Responsav
277 Responsave
296 Responsavel
308 ResponsavelP
370 ResponsavelPe
397 ResponsavelPel
434 ResponsavelPelpQ
448 ResponsavelPeloQ
451 ResponsavelPeloQu
508 ResponsavelPeloQue
580 ResponsavelPeloQueC
586 ResponsavelPeloQueCa
656 ResponsavelPeloQueCat
710 ResponsavelPeloQueCati
815 ResponsavelPeloQueCativ
958 ResponsavelPeloQueCativa
996 ResponsavelPeloQueCativas
1009 ResponsavelPeloQueCativas4
1081 ResponsavelPeloQueCativas43


Observamos, assim, que os operadores utilizados foram suficientes para resolver o problema rapidamente (menos de 2 segundos). Para comparação, caso a mesma senha fosse encontrada com busca exaustiva, seria necessário testar $62^{27} = 2.48\cdot 10^{48}$ possibilidades, ou seja, o número de caracteres possíveis elevado ao número de caixas. Em termos práticos isso é inviável: em testes aqui utilizando o `itertools.product`, a busca de uma senha de 4 caracteres já demora 2 segundos, enquanto uma senha de 5 caracteres causa erro por falta de memória.

### Conclusão

O algoritmo genético formado com operadores de criação de população, função objetivo, cruzamento e mutação pensados especificamente para o problema da senha de tamanho variável foi eficiente em encontrar uma solução através da busca com recompensa para indivíduos mais próximos da resposta.

### Referências

[1] EYAL WIRSANSKY. Hands-On Genetic Algorithms with Python: Applying genetic algorithms to solve real-world deep learning and artificial intelligence problems, 2020. Chapter 2: Understanding the Key Components of Genetic Algorithms, Crossover for ordered lists.